In [ ]:
import numpy as np
import pandas as pd
import f90nml
import os
import shutil
import warnings
import time
import xarray as xr
from datetime import timedelta
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [ ]:
def write_met_data(filename, input_data):

    fid = open(filename, "wt")

    fid.write("-" * 130 + "\n")

    fid.write(
        "{:^17s}|{:^12s}|{:^12s}|{:^10s}|{:^10s}|{:^11s}|{:^11s}|{:^15s}\n".format(
            "date/time",
            "windspeed",
            "temperature",
            "humidity",
            "pressure",
            "shortwave",
            "longwave",
            "precipitation",
        )
    )

    fid.write(
        "{:^17s}|{:^12s}|{:^12s}|{:^10s}|{:^10s}|{:^11s}|{:^11s}|{:^15s}\n".format(
            "yyyy mm dd hh mi",
            "m s{-1}",
            "K",
            " %",
            "Pa",
            "W m{-2}",
            "W m{-2}",
            "mm d{-1}",
        )
    )

    fid.write("-" * 130 + "\n")

    for i, date0 in enumerate(input_data.index):

        # print(i, date0.year)

        fid.write(
            "{:4d} {:02d} {:02d} {:02d} {:02d}  {:^12.3f} {:^12.3f} {:^10.3f} {:^10.1f} {:^11.3f} {:^11.3f} {:^15.2f}\n".format(
                date0.year,
                date0.month,
                date0.day,
                date0.hour,
                date0.minute,
                input_data["wind_speed"][i],
                input_data["air_temperature"][i],
                input_data["relative_humidity"][i],
                input_data["surface_pressure"][i],
                input_data["shortwave_downward"][i],
                input_data["longwave_downward"][i],
                input_data["precipitation"][i],
            )
        )

    fid.close()

In [ ]:
def mkdir(foldername, cleanup=False):
    if os.path.exists(foldername):
        if cleanup:
            os.system("rm " + os.path.join(foldername, "*"))
        pass
    else:
        os.makedirs(foldername)
        print("Creating " + foldername)
    return

In [ ]:
def regroup_365days(ncid, year=None):
    raw_data2 = ncid.convert_calendar("noleap")
    raw_data_group = (
        raw_data2.groupby(raw_data2.time.dt.dayofyear).mean().to_dataframe()
    )
    if year is None:
        raw_data_group.index = pd.date_range(
            ncid.time[0].values, periods=len(
                raw_data_group.air_temperature.values)
        )
    else:
        raw_data_group.index = pd.date_range(
            str(year) + "-01-01", periods=len(raw_data_group.air_temperature.values)
        )
    return raw_data_group

In [ ]:
Sitename = "Barrow"

rh_const = 50
ws_const = 4.0
ps_const = 101000
# 
# freq = "2D"
# first = "06-16"   # '05-18'
# last  = '09-28'
# dateoff  = '0D'

In [ ]:
# ==================

phase_shift = 90

dump_t365 = np.arange(1, 366)

tas = -9 + 15.0 * np.sin(2 * np.pi * (dump_t365 + phase_shift) / 365) + 273.15

# spinup_data['air_temperature'] = tas + 273.15

# ----

rsds = 115 + 221 / 2 * np.sin(2 * np.pi * (dump_t365 + phase_shift) / 365)

# ----

rh = rh_const / 100 + dump_t365 * 0
wdsp = ws_const + dump_t365 * 0

# ----

n_snow_days = sum(tas <= 273.15)
n_rain_days = sum(tas > 273.15)

idx_snw_days = tas * 0.0  # [spinup_data['air_temperature']<273.15]

idx_snw_days[tas <= 273.15] = 5 / 86400 / n_snow_days

idx_rain_days = tas * 0.0  # [spinup_data['air_temperature']<273.15]

idx_rain_days[tas > 273.15] = 60 / 86400 / n_snow_days

# ----

# tdew = 243.5*(np.log(rh)+((17.67*(tas - 273.15))/(243.5+tas - 273.15)))/(17.67-np.log(rh)-((17.67*(tas-273.15))/(243.5+tas - 273.15)))
# evap = 10.0 ** (11.40 - 2353.0 / (tdew + 273.15))

evap = rh * 611.2 * np.exp(17.67 * (tas - 273.15) / (tas - 29.65))

# svp = 611.2*exp(17.67*(temperature-273.15)/(temperature-29.65))

rlds = 1.08 * (1 - np.exp(-((0.01 * evap) ** (tas / 2016)))) * \
    5.670676e-8 * tas**4

# ----

# plt.plot(dump_t365, tas - 273.15)
# plt.plot(dump_t365, idx_snw_days)
# plt.plot(dump_t365, idx_rain_days)

plt.plot(dump_t365, rsds)
# plt.plot(dump_t365, rlds)


print(idx_rain_days.sum() * 86400, idx_snw_days.sum() * 86400)

print(rsds.min())

In [ ]:
df2 = pd.DataFrame(
    index=pd.date_range("1998-07-01", "1999-06-30"),
    columns=["TAS", "RSDS", "RH", "WSPD", "SNOW", "RAIN", "PREC"],
)

df2["air_temperature"] = tas.copy()
df2["shortwave_downward"] = rsds.copy()
df2["longwave_downward"] = rlds.copy()
df2["relative_humidity"] = rh_const + 0.0
df2["wind_speed"] = ws_const + 0.0
df2["surface_pressure"] = ps_const + 0.0
df2["SNOW"] = idx_snw_days.copy() * 86400
# df['RAIN'] = idx_rain_days.copy() / 1000
df2["precipitation"] = idx_snw_days * 86400 + idx_rain_days * 86400

# >>> Following lines are to insert 12-31 in the beginning >>>

add_onset_date = False

if add_onset_date:

    dump1232 = df2.iloc[-1]
    dump1232.name = pd.to_datetime(dump1232.name) - timedelta(days=365)
    dump1232

    df2 = pd.concat([dump1232.to_frame().T, df2])

# <<< END <<<

input_data = df2.copy()

In [ ]:
# soil_thickness = np.array([0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.3, 0.4, 0.4, 0.8, 0.8, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 10, 10, 10])

soil_thickness = np.loadtxt("soil_thickness.dat")

cum_soil_thickness = np.cumsum(soil_thickness)

soil_nodes = soil_thickness + 0.0

for i, d0 in enumerate(soil_thickness):

    if i == 0:
        soil_nodes[i] = np.round(d0 / 2.0, 5)
    else:
        soil_nodes[i] = np.round(
            cum_soil_thickness[i - 1] + 0.5 * (soil_thickness[i]), 5
        )

print(len(soil_thickness), sum(soil_thickness))

# soil_nodes.tolist()

In [ ]:
default_namelist = f90nml.read("README.namelist")

new_namelist = {
    "latitude": 71.2,
    "longitude": -156.25,
    "soil_type_index": 13,
    "vegetation_type_index": 3,
    "startdate": "199807010000",
    "enddate": "199906300000",
    "Skin_Temperature": 279.00,
    "canopy_water": 0.1,
    "forcing_timestep": 86400,
    "noahlsm_timestep": 600,
    "deep_soil_temperature": -9 + 273.15,
    "Soil_layer_thickness": soil_thickness.tolist(),
    "soil_temperature": (np.zeros(len(soil_thickness)) + 275.15).tolist(),
    "soil_moisture": (np.zeros(len(soil_thickness)) + 0.55).tolist(),
    "soil_liquid": (np.zeros(len(soil_thickness)) + 0.5).tolist(),
}

default_namelist["METADATA_NAMELIST"].update(new_namelist)

default_namelist.write(os.path.join("namelist.noahmp11"), force=True)

# default_namelist

In [ ]:
fid = open("namelist.noahmp11", "a")

fid.write("-" * 130 + "\n")

fid.write(
    "{:^17s}|{:^12s}|{:^12s}|{:^10s}|{:^10s}|{:^11s}|{:^11s}|{:^15s}\n".format(
        "date/time",
        "windspeed",
        "temperature",
        "humidity",
        "pressure",
        "shortwave",
        "longwave",
        "precipitation",
    )
)

fid.write(
    "{:^17s}|{:^12s}|{:^12s}|{:^10s}|{:^10s}|{:^11s}|{:^11s}|{:^15s}\n".format(
        "yyyy mm dd hh mi", "m s{-1}", "K", " %", "Pa", "W m{-2}", "W m{-2}", "mm d{-1}"
    )
)

fid.write("-" * 130 + "\n")

fid.write("<Forcing>\n")

for i, date0 in enumerate(input_data.index):

    # print(i, date0.year)

    fid.write(
        "{:4d} {:02d} {:02d} {:02d} {:02d} {:^17.10f} {:^17.10f} {:^17.10f} {:^17.10f} {:^17.10f} {:^17.10f} {:^17.10f} {:^17.10f}\n".format(
            date0.year,
            date0.month,
            date0.day,
            date0.hour,
            date0.minute,
            input_data["wind_speed"][i],
            0,
            input_data["air_temperature"][i],
            input_data["relative_humidity"][i],
            input_data["surface_pressure"][i] / 100,
            input_data["shortwave_downward"][i],
            input_data["longwave_downward"][i],
            input_data["precipitation"][i] / 86400,
        )
    )

fid.close()

In [ ]:
os.system("./src/driver.exe namelist.noahmp11")